## Calculate Future Energy Prices

Two strategies:

[1] Monthly average price from CBS + inflation rate (2% or 4%):   Future Price = [Current Price × (1 + inflation rate)^years into the future]

[2] Policy based scenarios 

Steps:

[1] Data collection for the current energy prices from CBS

[2] Inflation correction

[3] Determining energy scenarios based on KEV: scenario-based price pathways (e.g. moderate and high)


References:

CBS monthly average prices for gas and electricity: https://opendata.cbs.nl/statline/#/CBS/en/dataset/85592ENG/table?ts=1749022887872

PBL future scenarios for gas and electricity: https://www.pbl.nl/kev/publicaties ( pbl-2023-scenarios-voor-aardgas-en-elektriciteitsprijzen-voor-kleinverbruikers-in-de-periode-2025-2040_0)

Notes:

CBL prices include VAT

Total CBL prices are calculated based on the 'Variable delivery rate contract prices (Euro/m3)' and 'Energy tax (Euro/m3)' figures in CSV 

## Importing input data

In [1]:
import os

# importing sys
import sys

import tracemalloc

import numpy as np
import pandas as pd
from tqdm import tqdm

%matplotlib inline

tracemalloc.start()

In [2]:
# Step 1: Load the dataset
file_path_CBS = 'energy_prices_original.csv'  
df = pd.read_csv(file_path_CBS)

In [3]:
columns_to_remove = [
    'Natural gas/Variable delivery rate with price cap (Euro/m3)', 
    'Electricity/Variable delivery rate with price cap (Euro/kWh)', 
    'Electricity/Energy tax refund (Euro/year)'
]

df = df.drop(columns=columns_to_remove, errors='ignore')

In [4]:
#transport rate is per year in the original dataset, so it needs a conversion
df['Electricity/Transport rate (Euro/year)'] = df['Electricity/Transport rate (Euro/year)'] / 12
df['Natural gas/Transport rate (Euro/year)'] = df['Natural gas/Transport rate (Euro/year)'] / 12

#define annual consumption for electricity and natural gas (now i am using average household consumption)
df['Electricity/Annual consumption (kWh)'] = 3500 # average household consumption
df['Natural gas/Annual consumption (m3)'] = 1500 # average household consumption

# Step 2: Calculate the total cost for each energy type
df['Transport_rate_electricity_kwh'] = df['Electricity/Transport rate (Euro/year)'] / df['Electricity/Annual consumption (kWh)']
df['Transport_rate_natural_gas_m3'] = df['Natural gas/Transport rate (Euro/year)'] / df['Natural gas/Annual consumption (m3)']

In [5]:
# Step 3: Extract and compute prices
df['Total Gas Price (€/m³)'] = (
    df['Natural gas/Variable delivery rate contract prices (Euro/m3)'] +
    df['Natural gas/Energy tax (Euro/m3)'] + df['Transport_rate_natural_gas_m3']
)

df['Total Electricity Price (€/kWh)'] = (
    df['Electricity/Variable delivery rate contract prices (Euro/kWh)'] +
    df['Electricity/Energy tax (Euro/kWh)'] + df['Transport_rate_electricity_kwh']
)

In [6]:
print(df['Total Gas Price (€/m³)'].head())
print(df['Total Electricity Price (€/kWh)'].head())

0    1.407951
1    1.395251
2    1.356451
3    1.308751
4    1.304251
Name: Total Gas Price (€/m³), dtype: float64
0    0.340347
1    0.335147
2    0.323847
3    0.314447
4    0.311847
Name: Total Electricity Price (€/kWh), dtype: float64


## [1] CBS + inflation rate (2% or 4%)


In [26]:
inflation_rate = 0.02  # Example inflation rate of 2%
years = 5 # Example period of 5 years
df['Future Gas Price (€/m³)'] = df['Total Gas Price (€/m³)'] * (1 + inflation_rate) ** years
df['Future Electricity Price (€/kWh)'] = df['Total Electricity Price (€/kWh)'] * (1 + inflation_rate) ** years

In [27]:
average_gas_price = df['Future Gas Price (€/m³)'].mean()
average_electricity_price = df['Future Electricity Price (€/kWh)'].mean()

In [28]:
average_electricity_price, average_gas_price

(np.float64(0.33698583405622284), np.float64(1.481545174954464))

## [2] Using policy based scenarios  

In [59]:
# Load data
df_E = pd.read_csv('Scenarios_Electricity.csv')
df_G = pd.read_csv('Scenarios_Gas.csv')

# Ensure year is numeric
df_E['Year'] = pd.to_numeric(df_E['Year'], errors='coerce')
df_G['Year'] = pd.to_numeric(df_G['Year'], errors='coerce')

# Clean column names by stripping whitespace and removing text like "(excl. VAT)"
def clean_columns(df):
    df.columns = df.columns.str.replace(r"\(.*?\)", "", regex=True)
    df.columns = df.columns.str.strip().str.lower()
    return df

df_E = clean_columns(df_E)
df_G = clean_columns(df_G)

# Apply VAT
def add_vat(df, vat_rate=0.21):
    df = df.copy()
    for col in df.columns:
        if col != 'year':
            df[col] = pd.to_numeric(df[col], errors='coerce')
    numeric_cols = df.select_dtypes(include='number').columns.drop('year', errors='ignore')
    df[numeric_cols] = df[numeric_cols] * (1 + vat_rate)
    return df

df_E = add_vat(df_E)
df_G = add_vat(df_G)

# Function to find column with keyword like 'central'
def find_column_with_keyword(df, keyword):
    for col in df.columns:
        if keyword in col:
            return col
    return None

In [60]:
# User input
year = 2025
price = 'central'  # 'central', 'higher', or 'lower'

# Find columns
col_E = find_column_with_keyword(df_E, price)
col_G = find_column_with_keyword(df_G, price)

# Print results
if col_E:
    row_E = df_E[df_E['year'] == year][['year', col_E]]
    if not row_E.empty:
        print(f"Electricity | Year: {year} | Price type: {price}\n{row_E.to_string(index=False)}")
    else:
        print(f"No electricity data found for year {year}.")
else:
    print("Electricity price column not found.")

if col_G:
    row_G = df_G[df_G['year'] == year][['year', col_G]]
    if not row_G.empty:
        print(f"\nGas | Year: {year} | Price type: {price}\n{row_G.to_string(index=False)}")
    else:
        print(f"No gas data found for year {year}.")
else:
    print("Gas price column not found.")

Electricity | Year: 2025 | Price type: central
 year  central prices
 2025          0.2541

Gas | Year: 2025 | Price type: central
 year  central prices
 2025           1.331
